In [1]:
# Import comet_ml at the top of your file
import comet_ml
from pytorch_lightning.loggers import CometLogger
import sys 
sys.path.append('..')



In [2]:
import os
from src.datasets.mimic_cxr_dataset import MIMICCXRDataModule
from pytorch_lightning import Trainer, seed_everything
import torchvision.transforms as T
import torch
# sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
seed_everything(42)


Global seed set to 42


42

In [3]:

augmentations = {'train':
    T.Compose(
    [
        T.Resize((224, 224)),
        #T.RandomApply([T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.8),
        #T.RandomGrayscale(p=0.2),
        #T.GaussianBlur(kernel_size=9),
        T.ToTensor(),
    ]),
    'val':
    T.Compose(
    [
        T.Resize((224, 224)),
        #T.RandomApply([T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.8),
        #T.RandomGrayscale(p=0.2),
        #T.GaussianBlur(kernel_size=9),
        T.ToTensor(),
    ])
}


In [4]:
BATCH_SIZE = 8
NUM_EPOCHS = 200
LIMIT_NUM_SAMPLES = 16
NUM_DATA_WORKERS  = 0
ONLY_IMAGES = False

MIMIC_CXR_DCM_PATH = '/Users/caghankoksal/Desktop/development/Flamingo-playground/physionet.org/files/mimic-cxr/2.0.0/files/'
MIMIC_CXR_JPG_PATH = '/Users/caghankoksal/Desktop/development/physionet.org/files/mimic-cxr-jpg/2.0.0/files/'
SPLIT_PATH = '/Users/caghankoksal/Desktop/SS2022/mlmi-vqa/data/external/'



In [5]:

mimic_datamodule = MIMICCXRDataModule(MIMIC_CXR_DCM_PATH, MIMIC_CXR_JPG_PATH, 
                                      transforms=augmentations, only_images=False, batch_size=BATCH_SIZE,
                                      limit_num_samples=LIMIT_NUM_SAMPLES, num_data_workers=NUM_DATA_WORKERS,
                                      tokenizer="gpt2",image_type="jpg", split_path=SPLIT_PATH, preprocessed=True)
train_loader = mimic_datamodule.train_dataloader()
val_loader = mimic_datamodule.val_dataloader()

In [6]:
mimic_datamodule.train_dataset[0]

{'image': tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0431, 0.0471, 0.0510],
          [0.0000, 0.0000, 0.0000,  ..., 0.0706, 0.0745, 0.0784],
          [0.0000, 0.0000, 0.0000,  ..., 0.0627, 0.0667, 0.0667],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.4902, 0.4784, 0.4549],
          [0.0000, 0.0000, 0.0000,  ..., 0.4980, 0.4745, 0.4510],
          [0.0000, 0.0000, 0.0000,  ..., 0.5137, 0.4902, 0.4627]],
 
         [[0.0000, 0.0000, 0.0000,  ..., 0.0431, 0.0471, 0.0510],
          [0.0000, 0.0000, 0.0000,  ..., 0.0706, 0.0745, 0.0784],
          [0.0000, 0.0000, 0.0000,  ..., 0.0627, 0.0667, 0.0667],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.4902, 0.4784, 0.4549],
          [0.0000, 0.0000, 0.0000,  ..., 0.4980, 0.4745, 0.4510],
          [0.0000, 0.0000, 0.0000,  ..., 0.5137, 0.4902, 0.4627]],
 
         [[0.0000, 0.0000, 0.0000,  ..., 0.0431, 0.0471, 0.0510],
          [0.0000, 0.0000, 0.0000,  ..., 0.0706, 0.0745, 0.0784],
          [0.0000, 0.0000, 0.00

In [7]:
len(mimic_datamodule.train_dataset)

16

In [8]:
len(mimic_datamodule.validation_dataset)

16

In [9]:
len(mimic_datamodule.test_dataset)

16

In [10]:
len(train_loader)

2

In [11]:
for batch in train_loader:
    print("Image shape : ",batch["image"].shape)
    print(batch["text"][0]) # First sample in the batch
    print("Input Ids shape : ",batch["input_ids"].shape)
    print("Targets shape : ",batch["targets"].shape)
    break

Image shape :  torch.Size([8, 3, 224, 224])
<|endoftext|> <image> Output: FINAL REPORT EXAMINATION:  CHEST (PORTABLE AP) INDICATION:   year old man with concern for pulmonary renal syndrome/pna  // acute process, worsening TECHNIQUE:  CHEST (PORTABLE AP) COMPARISON:   IMPRESSION: Cardiomediastinal silhouette demonstrate mild dilatation as compared to the previous study, diffuse. There is also vascular enlargement, consistent with pulmonary edema.  New right basal opacity concerning for interval development of atelectasis or aspiration is demonstrated. <EOC>
Input Ids shape :  torch.Size([8, 1, 256])
Targets shape :  torch.Size([8, 1, 256])


In [12]:
from src.models.multimodal.flamingo_module import FlamingoModule
import pytorch_lightning as pl

In [13]:
VOCAB_SIZE_OF_TOKENIZER = 50257 # mimic_datamodule.train_dataset.tokenizer.vocab_size
LANGUAGE_MODEL = 'gpt2'
NUM_TOKENS = VOCAB_SIZE_OF_TOKENIZER +3 if LANGUAGE_MODEL=="gpt2" else 31092
FLAMINGO_EMBED_DIM = 768
DEPTH = 12
NUM_HEADS = 8
ATT_HEAD_DIM = 64
CROOS_ATT_EVERY=3
MEDIA_TOKEN_ID = mimic_datamodule.train_dataset.tokenizer.all_special_ids[mimic_datamodule.train_dataset.tokenizer.all_special_tokens.index('<image>')]
PERCEIVER_NUM_LATENTS = 64
PERCEIVER_DEPTH = 2
IMAGE_ENCODER = "clip"
PRETRAINED_CLIP_PATH = '/Users/caghankoksal/Desktop/development/PubMedCLIP_ViT32.pth'
PRETRAINED_GPT2_PATH = "/Users/caghankoksal/Desktop/development/TransformerPlay/gpt2-pytorch_model.bin"

print(  "VOCAB_SIZE_OF_TOKENIZER : ",VOCAB_SIZE_OF_TOKENIZER, "\n"
        "LANGUAGE_MODEL : ",LANGUAGE_MODEL, "\n"
        "NUM_TOKENS : ",NUM_TOKENS, "\n"
        "FLAMINGO_EMBED_DIM : ",FLAMINGO_EMBED_DIM, "\n"
        "DEPTH : ",DEPTH, "\n"
        "NUM_HEADS : ",NUM_HEADS, "\n"
        "ATT_HEAD_DIM : ",ATT_HEAD_DIM, "\n"
        "CROOS_ATT_EVERY : ",CROOS_ATT_EVERY, "\n"
        "MEDIA_TOKEN_ID : ",MEDIA_TOKEN_ID, "\n"
        "PERCEIVER_NUM_LATENTS : ",PERCEIVER_NUM_LATENTS, "\n"
        "PERCEIVER_DEPTH : ",PERCEIVER_DEPTH, "\n"
        "IMAGE_ENCODER : ",IMAGE_ENCODER, "\n"
        "PRETRAINED_CLIP_PATH : ",PRETRAINED_CLIP_PATH, "\n"
        "PRETRAINED_GPT2_PATH : ",PRETRAINED_GPT2_PATH, "\n")

VOCAB_SIZE_OF_TOKENIZER :  50257 
LANGUAGE_MODEL :  gpt2 
NUM_TOKENS :  50260 
FLAMINGO_EMBED_DIM :  768 
DEPTH :  12 
NUM_HEADS :  8 
ATT_HEAD_DIM :  64 
CROOS_ATT_EVERY :  3 
MEDIA_TOKEN_ID :  50258 
PERCEIVER_NUM_LATENTS :  64 
PERCEIVER_DEPTH :  2 
IMAGE_ENCODER :  clip 
PRETRAINED_CLIP_PATH :  /Users/caghankoksal/Desktop/development/PubMedCLIP_ViT32.pth 
PRETRAINED_GPT2_PATH :  /Users/caghankoksal/Desktop/development/TransformerPlay/gpt2-pytorch_model.bin 



In [14]:
hyperparams = {
    'pretrained_clip_path': PRETRAINED_CLIP_PATH,
    'warmup_steps': 569,
    'num_tokens': NUM_TOKENS,
    'dim': FLAMINGO_EMBED_DIM,
    'depth': DEPTH,
    'num_heads': NUM_HEADS,
    'dim_head': ATT_HEAD_DIM,
    'cross_attn_every': CROOS_ATT_EVERY,
    'media_token_id': MEDIA_TOKEN_ID,
    'perceiver_num_latents': PERCEIVER_NUM_LATENTS,
    'perceiver_depth': PERCEIVER_DEPTH,
    'image_encoder': IMAGE_ENCODER,
    'language_model': LANGUAGE_MODEL,
    'pretrained_gpt2_path': PRETRAINED_GPT2_PATH,
}

In [15]:
model = FlamingoModule(**hyperparams)

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/61/9c2llh9n2pjb81c4dmhmb67w0000gn/T/tmp5mh91vu6
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/61/9c2llh9n2pjb81c4dmhmb67w0000gn/T/tmp5mh91vu6/_remote_module_non_sriptable.py


Pretrained clip is being loaded
Flamingo is being initialized with  gpt2  as language model
GPT 2 Weights are loading...
Loaded GPT2 weights and Embeddings num_weights loaded :  156


In [16]:
iterator = iter(train_loader)

In [17]:
batch = next(iterator)
out, attns = model(batch, return_attn = True)


In [18]:
model(batch)


tensor([[[ 0.1584, -0.3243,  0.0264,  ..., -0.2829, -0.5910, -0.0099],
         [-0.3828, -0.0156, -0.4264,  ..., -0.8193, -0.1046, -0.1572],
         [-0.4204,  0.7016,  0.0097,  ..., -0.2313,  0.3466,  0.1772],
         ...,
         [ 0.2086,  0.0754, -0.4519,  ...,  0.1039,  0.0590,  0.1050],
         [-0.0104,  0.1238, -0.1079,  ..., -0.0618,  0.2761, -0.0398],
         [ 0.3627,  0.2320, -0.0376,  ...,  0.2112,  0.1849,  0.1083]],

        [[-0.4783,  0.7416,  0.2245,  ..., -0.3303,  0.0104,  0.4521],
         [-0.5691, -0.1834, -0.1711,  ..., -1.0878,  0.3459, -0.4896],
         [-0.5099,  0.6013,  0.1814,  ..., -0.2899,  0.3962,  0.0820],
         ...,
         [-0.1608,  0.3808, -0.1850,  ..., -0.5502, -0.4207,  0.1371],
         [ 0.0264,  0.4373, -0.0424,  ..., -0.4474, -0.2347,  0.0716],
         [ 0.2198,  0.3883,  0.2171,  ..., -0.6124, -0.4146,  0.1145]],

        [[-0.3992,  0.6790,  0.1200,  ..., -0.4232, -0.1586,  0.4711],
         [-0.7167, -0.4431, -0.1101,  ..., -0

In [19]:
import numpy as np
import cv2
from torchvision.transforms import transforms
import matplotlib.pyplot as plt

In [20]:
# Attention Maps
batch = next(iterator)
out, attns = model(batch, return_attn = True)
img = batch["image"]
# Stack attention maps
att_mat = torch.stack(attns)
att_mat = att_mat.squeeze(1)
# Average the attention weights across all heads.
#att_mat = torch.mean(att_mat, dim=0)

# Take the Last attention map
att_mat = att_mat[-1,:,:]
#Use Classification Token's attention
cur_attn_map = att_mat[0,1:]

grid_size = int(np.sqrt(cur_attn_map.shape[0])) # 49, 49
mask = cur_attn_map.reshape(grid_size, grid_size).detach().numpy()

im = transforms.ToPILImage()(img.squeeze(0))
# Maybe use interpolate instead of resize
mask2 = cv2.resize(mask / mask.max(), im.size)[..., np.newaxis]
result = (torch.tensor(mask2[:,:,0]) * img[0][0])
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 16))

ax1.set_title('Original')
ax2.set_title('Attention Map')
_ = ax1.imshow(im)
_ = ax2.imshow(result)

RuntimeError: shape '[7, 7]' is invalid for input of size 2450

In [ ]:
batch = next(iterator)
out, attns = model(batch, return_attn = True)
# Attention Maps
img = batch["image"]
# Stack attention maps


In [ ]:
from src.preprocess.visualize_attention import plot_attention_maps

In [ ]:
import numpy as np
import cv2
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
import torch
from torch import nn

def plot_attention_maps2(attns, img, num_heads=12):

    
    fig, ax = plt.subplots(num_heads, 4)
    fig.set_size_inches(18.5, 70)


    # Stack attention maps
    att_maps = torch.stack(attns)
    att_maps = att_maps.squeeze(1)
    # Average the attention weights across all heads.
    #att_mat = torch.mean(att_mat, dim=0)


    for j in range(num_heads):
        #fname = os.path.join(args.output_dir, "attn-head" + str(j) + ".png")
        #plt.imsave(fname=fname, arr=attentions[j], format='png')
        #print(f"{fname} saved.")

        # Take the Last attention map
        cur_attention_head = att_maps[j,:,:]
        #Use Classification Token's attention
        cur_attn_map = cur_attention_head[0,1:]

        grid_size = int(np.sqrt(cur_attn_map.shape[0])) # 49, 49
        mask = cur_attn_map.reshape(grid_size, grid_size).detach().numpy()

        im = transforms.ToPILImage()(img.squeeze(0))
        # Maybe use interpolate instead of resize
        
        interpol_mask = nn.functional.interpolate(torch.tensor(mask).unsqueeze(0).unsqueeze(1), scale_factor=32, mode="nearest")
        interpol_mask = interpol_mask[0,0,:,:].unsqueeze(2)
        result_interpol = (torch.tensor(interpol_mask[:,:,0]) * img[0][0])
        
        resize_mask = cv2.resize(mask / mask.max(), im.size)[..., np.newaxis]
        result_resize = (torch.tensor(resize_mask[:,:,0]) * img[0][0])


        ax[j][0].imshow(im)
        ax[j][0].set_title("Orginal")
        
        
        ax[j][1].imshow(im)
        #ax[j][1].imshow(mask, cmap='hot', alpha=0.7)
        ax[j][1].imshow(resize_mask, cmap='hot', alpha=0.7)
        ax[j][1].set_title("Head Attention" + str(j))
        
        ax[j][2].imshow(result_resize)
        ax[j][2].set_title("Head result_resize Attention" + str(j))

        ax[j][3].imshow(resize_mask)
        ax[j][3].set_title("Head resize_mask Attention" + str(j))

    
    plt.show()

In [ ]:
plot_attention_maps2(attns, img, num_heads=12)

In [ ]:
batch = next(iterator)
out, attns = model(batch, return_attn = True)
# Attention Maps
img = batch["image"]
# Stack attention maps


In [ ]:
# To account for residual connections, we add an identity matrix to the
# attention matrix and re-normalize the weights.
residual_att = torch.eye(att_mat.size(1))
aug_att_mat = att_mat + residual_att
aug_att_mat = aug_att_mat / aug_att_mat.sum(dim=-1).unsqueeze(-1)

# Recursively multiply the weight matrices
joint_attentions = torch.zeros(aug_att_mat.size())
joint_attentions[0] = aug_att_mat[0]
im = transforms.ToPILImage()(img.squeeze(0))
for n in range(1, aug_att_mat.size(0)):
    joint_attentions[n] = torch.matmul(aug_att_mat[n], joint_attentions[n-1])
print("JA shape: ",joint_attentions.shape)
# Attention from the output token to the input space.

grid_size = int(np.sqrt(aug_att_mat.size(-1)))
mask = joint_attentions[0, 1:].reshape(grid_size, grid_size).detach().numpy()
mask = cv2.resize(mask / mask.max(), im.size)[..., np.newaxis]
result = (mask * im).astype("uint8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 16))

ax1.set_title('Original')
ax2.set_title('Attention Map')
_ = ax1.imshow(im)
_ = ax2.imshow(result)



COMET_API_KEY = "F2L19mQwKXSoeF1IYEDA2AeHD",
PROJECT_KEY = "flamingo-playground",
import os
from pytorch_lightning.loggers import CometLogger

comet_logger = CometLogger(
    api_key= "F2L19mQwKXSoeF1IYEDA2AeHD",
    project_name="flamingo-gpt2")


In [ ]:
from pytorch_lightning.callbacks import LearningRateMonitor
lr_monitor = LearningRateMonitor(logging_interval='step')

In [ ]:
trainer = pl.Trainer(max_epochs=6,
                     accelerator="cpu", devices=1,
                     callbacks=[lr_monitor],
                     log_every_n_steps=1,
                      )



In [ ]:
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

In [ ]:
!tensorboard --logdir=lightning_logs

In [ ]:
from pytorch_lightning.utilities.model_summary import LayerSummary

In [ ]:
summary = LayerSummary(model.flamingo_palm.token_emb)
summary.num_parameters

In [ ]:
 from transformers import AutoTokenizer


In [ ]:
# INFERENCE

In [ ]:
import torch
import torchvision
import torchvision.transforms as T
from PIL import Image
from transformers import GPT2Tokenizer
import numpy as np

from torch import nn as nn
import torch.nn.functional as F
def generate(image, context, ntok=20):
    for _ in range(ntok):
        out = model({'image': image,'input_ids': context })
        logits = out[:, -1, :]
        indices_to_remove = logits < torch.topk(logits, 10)[0][..., -1, None]
        logits[indices_to_remove] = np.NINF
        next_tok = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1).squeeze(1)
        context = torch.cat([context, next_tok.unsqueeze(-1)], dim=-1)
    return context


tokenizer = mimic_datamodule.train_dataset.tokenizer

In [ ]:
ckpt_folder_path = "/Users/caghankoksal/Desktop/logs_from_cluster/lightning_logs_flamingo-gpt2/"

In [ ]:
os.listdir(ckpt_folder_path)

In [ ]:
model_folder_name = '9_da477150ff4a4f50866683712eb8c707'

In [ ]:
os.listdir(os.path.join(ckpt_folder_path,model_folder_name))

In [ ]:
os.listdir(os.path.join(ckpt_folder_path,model_folder_name, "checkpoints"))

In [ ]:
CHECKPOINT_PATH = os.path.join(ckpt_folder_path,model_folder_name,"checkpoints",'epoch=176-step=49206.ckpt')
model.load_state_dict(torch.load(CHECKPOINT_PATH,map_location=torch.device('cpu'))["state_dict"])
model.eval()

In [ ]:
LIMIT_NUM_SAMPLES = None

In [ ]:
MIMIC_CXR_DCM_PATH = '/Users/caghankoksal/Desktop/development/Flamingo-playground/physionet.org/files/mimic-cxr/2.0.0/files/'
MIMIC_CXR_JPG_PATH = '/Users/caghankoksal/Desktop/development/physionet.org/files/mimic-cxr-jpg/2.0.0/files/'
mimic_datamodule = MIMICCXRDataModule(MIMIC_CXR_DCM_PATH, MIMIC_CXR_JPG_PATH, 
                                      transforms=augmentations, only_images=False, batch_size=1,
                                      limit_num_samples=LIMIT_NUM_SAMPLES, num_data_workers=0,
                                      tokenizer="gpt2",image_type="jpg")
val_loader = mimic_datamodule.val_dataloader()

In [ ]:
load_val = iter(val_loader)


In [ ]:
batch = next(load_val)
print(batch["text"]),
img = T.ToPILImage()(batch["image"][0])
img

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output:')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output: FINDINGS: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Question: Is there a penumothorax? If yes, explain why? Answer: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
batch = next(load_val)
print(batch["text"]),
img = T.ToPILImage()(batch["image"][0])
img

In [ ]:
context   = torch.tensor([tokenizer.encode('<|endoftext|> <image> Output: ')])
out = generate(batch["image"], context, ntok=180)
tokenizer.decode(out[0])

In [ ]:
# VISUALIZE VIT ATTENTION

In [ ]:
# BEAM SEARCH

In [ ]:
import operator
import torch
import torch.nn as nn
import torch.nn.functional as F
from queue import PriorityQueue
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SOS_token = 0
EOS_token = 1
MAX_LENGTH = 50


In [ ]:
out.shape

In [ ]:
class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward

In [ ]:
def beam_decode(target_tensor, decoder_hiddens, encoder_outputs=None):
    '''
    :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
    :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
    :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
    :return: decoded_batch
    '''

    beam_width = 10
    topk = 1  # how many sentence do you want to generate
    decoded_batch = []

    # decoding goes sentence by sentence
    for idx in range(target_tensor.size(0)):
    
        decoder_hidden = decoder_hiddens[:, idx, :].unsqueeze(0)
        encoder_output = encoder_outputs[:,idx, :].unsqueeze(1)

        # Start with the start of the sentence token
        decoder_input = torch.LongTensor([[SOS_token]], device=device)

        # Number of sentence to generate
        endnodes = []
        number_required = min((topk + 1), topk - len(endnodes))

        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1

        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000: break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    break
                else:
                    continue

            # decode for one step using decoder
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_width)
            nextnodes = []

            for new_k in range(beam_width):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                # increase qsize
            qsize += len(nextnodes) - 1

        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(topk)]

        utterances = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):
            utterance = []
            utterance.append(n.wordid)
            # back trace
            while n.prevNode != None:
                n = n.prevNode
                utterance.append(n.wordid)

            utterance = utterance[::-1]
            utterances.append(utterance)

        decoded_batch.append(utterances)

    return decoded_batch